In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
import requests, json, csv

In [ ]:
def load_json_prop(json, prop):
    value=0
    for uf in json:
        value += uf[prop]
    return value

In [ ]:
def load_data():
    data = []

    i = 0
    for month in range(3, 9):
        for date in range(1, 32):
            i += 1
            # print("Collecting " + str(date) + "/" + str(month))
            response = requests.get("https://covid19-brazil-api.now.sh/api/report/v1/brazil/2020" + f"{month:02d}" + f"{date:02d}")
            if response.status_code != 200:
                continue
            responseJson = json.loads(response.text)["data"]
            cases = load_json_prop(responseJson, "cases")
            if cases == 0:
                continue
            row = []
            row.append(month)
            row.append(date)
            row.append(cases)
            row.append(i)
            data.append(row)

    return data

In [ ]:
data = load_data()

with open('database.csv', mode='w') as fl:
    fl = csv.writer(fl, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    fl.writerow(['Month', 'Day', 'Cases', 'Time'])

    for evidences in data:
        fl.writerow(evidences)

In [ ]:
cases = pd.read_csv("database.csv")
cases

In [ ]:
cases.plot.scatter(x="Time", y="Cases")

In [ ]:
model = RidgeCV()
evidences = cases[["Time"]]
labels = cases["Cases"]
train_x, test_x, train_y, test_y = train_test_split(evidences, labels)
model.fit(train_x, train_y)
predicts = model.predict(test_x)
# avaliar o acerto das predicoes